<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2022/ex05notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2022 ex05notebookA

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2022

---
## データの準備
---

このセクションでは，この notebook で使うデータの準備をします．

独立行政法人統計センターが作成している 教育用標準データセット(SSDSE) の中の「SSDSE-家計消費」というデータを使います．

「SSDSE-家計消費」は，47都道府県の県庁所在市で調査された家計消費に関する226項目の数値から成るデータです．「二人以上の世帯の1世帯あたりの焼き肉（外食）への年間支出金額」みたいな数値が含まれています．

データの詳細はこちら: https://www.nstac.go.jp/sys/files/kaisetsu-C-2022.pdf

このセクションのコードの作成にあたっては，データの前処理の部分でこちらを参考にさせていただきました： https://studiolab.sagemaker.aws/import/github/icoxfog417/mlnote-note/blob/main/notebooks/chapter10_answer.ipynb

以下のコードを上から順に実行しましょう

In [ ]:
# いろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

In [ ]:
##### CSV ファイルを読み込む #####
URL = 'https://www.nstac.go.jp/sys/files/SSDSE-C-2022.csv'
df_raw = pd.read_csv(URL, encoding='cp932') # 文字コードが UTF8 ではなく ShiftJIS なので，encoding オプションを指定して変換
df_raw.head(10) # 最初の10行を表示

In [ ]:
#####  データの前処理 step1  #####

# df_raw の見出し行に記されたコードと，1行目に記されたその項目名との対応表を作り，項目名の行を削除
code = df_raw.iloc[0, :]
print('##### コードと項目名の対応')
print(code)
df1 = df_raw.drop(0, axis=0)

# 'SSDSE-C-2022'列の都市コードと'Prefecture'列の都道府県名，'City'列の県庁所在地名を整理
city = df1.iloc[:, :3]
print()
print('##### 都市コード，都道府県名，県庁所在地名')
print(city.head(5))
df1 = df1.drop(['Prefecture', 'City'], axis=1)
df1 = df1.set_index('SSDSE-C-2022')

df1 = df1.drop('R00000', axis=0) # 'R00000'列は「全国」のデータなのでこれを削除
df1 = df1.astype('float32') # 値を浮動小数点型に変換

print()
df1.head(5)

In [ ]:
#####  データの前処理 step2  #####

# 一人あたりの外食支出金額のデータを作成
df2 = pd.DataFrame()

# コードが 'LB12' で始まる列が，様々な品目の外食支出額．ただし，1世帯あたりの金額
columns = {}
for c in df1.columns:
    if c.startswith('LB12') and c != 'LB12':
        #print(c, code[c])
        columns[c] = code[c]
        df2[c] = df1[c]

# 列名をコードから項目名に変更
df2 = df2.rename(columns=columns)

# 都道府県ごとの平均世帯人数で割って一人あたりの金額にする
df2 = df2.divide(df1['LA03'], axis=0)

# 県庁所在地名の列を追加
df2 = pd.merge(city.loc[:, ['SSDSE-C-2022', 'City']], df2, left_on='SSDSE-C-2022', right_index=True)
df2 = df2.reset_index(drop=True)

df_feature = df2

# DataFrame 表示時の小数部の表示桁数
pd.options.display.precision = 1

# 最初の5件
df_feature.head(5)

----
## 主成分分析 (1)
----

**主成分分析** (Principal Component Analysis, PCA) は，多変量（多次元）のデータが与えられたときに，そのデータが含む情報をなるべく損なわずにより低次元のデータに変換するための多変量解析手法です．
$D$個の変数から成る（$D$次元の）データが与えられたときに，それを$H \leq D$ 個の変数で表す（$H$次元に変換する）ことを目的とします．この変換によってデータの次元数が削減されますので，このような操作は **次元削減** (Dimensionaliry Reduction) と呼ばれることもあります．主成分分析は，次元削減のための手法の一つです．

---
### やりたいこと



In [ ]:
# 13種類の支出金額ならべた配列を作る
X = df_feature.drop(columns=['SSDSE-C-2022', 'City']).to_numpy()
print(X[:5, :])
print(X.shape)

上でやっていた準備作業では，47都道府県の県庁所在市の13種類の外食支出金額（ひとりあたり，単位は円）の数値が得られました．このデータでは47の市ごとにその特徴が13個の変数で表されていますので，13次元のデータといえます．
これだけ変数の数が多いと，それらの数値を眺めただけで各市の傾向を理解するのは困難です．そこで，「このようなデータをもっと低次元で（少ない数の変数で）表す」ことを考えます．主成分分析は，このようなことを行うための手法です．




次のセルを実行すると，上記のデータに主成分分析を適用して，13次元のデータを2次元に変換したものが得られます．

<!---
ここでは [scikit-learn](https://scikit-learn.org/) という Python 向けの機械学習ライブラリを使っています．scikit-learn については授業の範囲外ですので，コードの中身の理解は必須ではありません．
cf. [sklearn.decomposition.PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)
--->
ここでは，[statsmodels](https://www.statsmodels.org/) という Python 向け統計分析ライブラリを使っています．
cf. [statsmodels.multivariate.pca.PCA](https://www.statsmodels.org/dev/generated/statsmodels.multivariate.pca.PCA.html)

In [ ]:
# 主成分分析を利用して X を2次元に次元削減した Y を得る
from statsmodels.multivariate.pca import PCA
pca = PCA(X, ncomp=2, standardize=False, demean=True, normalize=False)
Y = pca.scores
print(Y[:5, :])
print(Y.shape)

`X` は13次元のデータだったので `shape` が `(47, 13)` でしたが，それを変換して得られた `Y` の `shape` は `(47, 2)` です．各市の特徴を2つの数値で表したものとなっています．

2次元のデータなので，散布図に描けます．描いてみるとこんなん：

In [ ]:
# Y の散布図
fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(Y[:, 0], Y[:, 1])
ax.scatter(Y[24:29, 0], Y[24:29, 1], label='kinki5')
ax.scatter(Y[ 1: 7, 0], Y[ 1: 7, 1], label='tohoku6')
ax.set_xlim(-15000, 15000)
ax.set_ylim(-15000, 15000)
ax.set_aspect('equal')
ax.axhline(0, color='gray') # x軸
ax.axvline(0, color='gray') # y軸
ax.legend()
for n in range(len(Y)):
    plt.annotate(f'{n}', (Y[n, 0]+300, Y[n, 1]+00))
plt.show()

横軸が `Y` の0列目の値，縦軸が 1 列目の値です．各点に付された番号は，行番号（0から始まる）に対応しています．

これを見ると，ほげ市とふが市は点が近いところにある → 似た傾向にありそうだ，といったことを読み取れそうですね．また，ちゃんとした説明は後にしますが，得られた値を見て各市の特徴をつかむことも容易になります．

この図では，わかりやすくするために，近畿5府県と東北6県の点の色を変えてみています．それぞれ，元の13次元のデータはこんなんです：

In [ ]:
# 近畿5府県
df_feature.iloc[24:29, :]

In [ ]:
# 東北6県
df_feature.iloc[1:7, :]

In [ ]:
# お好きなところをどぞ
df_feature.iloc[0, :]  # 行番号で
#df_feature.query('City == "札幌市"')  # City 列の値で

ここでは13次元のデータを2次元にしたので散布図を描いてみせましたが，いつでもどんなデータでも2次元にする，という話ではないことに注意してください．13次元のデータを4次元にしたり，1万次元のデータを10次元にしたり，データの性質と分析の目的によって次元数は様々です．

---
### 1次元にする場合の主成分分析

↑に書いたことをもう一度書いておきます：

> **主成分分析** (Principal Component Analysis, PCA) は，多変量（多次元）のデータが与えられたときに，そのデータが含む情報をなるべく損なわずにより低次元のデータに変換するための多変量解析手法です．
$D$個の変数から成る（$D$次元の）データが与えられたときに，それを$H \leq D$ 個の変数で表す（$H$次元に変換する）ことを目的とします．この変換によってデータの次元数が削減されますので，このような操作は **次元削減** (Dimensionaliry Reduction) と呼ばれることもあります．主成分分析は，次元削減のための手法の一つです．


ここでは，まずは $H = 1$ の場合について考えます．

#### 考え方

主成分分析では，「データが含む情報をなるべく損なわずにより低次元のデータに変換する」ということを，「**変換後のデータの分散がなるべく大きくなるように（元のデータの分散を保つように）する**」ということで達成しようとします．
ここでは，元のデータの次元数 $D = 2$ の場合を例として，これを1次元に次元削減する場合を図に描きながら考えてみましょう．



下図の左は，とある2次元のデータを散布図に描いたものです．二つの破線はそれぞれ，横軸と縦軸の値の平均を表しています．
このデータの値を，それらの平均が $\mathbf{0}$ になるように平行移動させると，散布図は下図の右のようになります．
以降，このように平均が $\mathbf{0}$ になるように平行移動させたデータの集合を 
$$
\{ \mathbf{x}_n \in {\cal R}^{D} | n = 1, 2, \ldots, N\}
$$
と表すことにします．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/pcafig01.png">


このようなデータを1次元にする線形変換は，$D$次元ベクトル $\mathbf{u} \in {\cal R}^{D}$ をひとつ定めて，下図のように$D$次元ベクトル $\mathbf{x}$ を1つの実数値 $y$ に変換する操作と考えられます．


<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/pcafig02.png">

この図の左の赤い直線は，ベクトル $\mathbf{u}$ の向きを表しています．個々のデータ $\mathbf{x}$ に対して，「その点がこの赤い直線上に落とす影」を考えて，この直線上でのその影の位置を $y$ とおくと，右図に示すように

$$
y = \Vert\mathbf{x}\Vert\cos\theta
$$

となります．$\theta$ は $\mathbf{x}$ と $\mathbf{u}$ の成す角，$\Vert\mathbf{x}\Vert$ は $\mathbf{x}$ の大きさです．
$\mathbf{u}\cdot\mathbf{x} = \Vert\mathbf{u}\Vert\Vert\mathbf{x}\Vert\cos\theta$ であることから，上の式は，次のように書き換えられます．

$$
y = \frac{\mathbf{u}\cdot\mathbf{x}}{\Vert\mathbf{u}\Vert}
$$

与えられたデータに対して，ベクトル $\mathbf{u}$ を適当に定めてやれば，この式によって $D$ 次元のデータを1つの実数値に変換できます．

ここで，ベクトル $\mathbf{u}$  として，向きが同じで大きさのみ異なる二つの候補があったとしてみましょう．その場合，得られる $y$ の値はどちらを用いても同じとなります．つまり，ここで考えている変換では，ベクトル $\mathbf{u}$ の向きのみが重要ということになります．
そのため，ベクトル $\mathbf{u}$ の大きさは $1$ に限定して考えることができます．この場合，$\Vert\mathbf{u}\Vert = 1$ ですので，上の式はより簡単に

$$
y = \mathbf{u}\cdot\mathbf{x}
$$

と表せます．

以上より，大きさ$1$の$D$次元ベクトル $\mathbf{u}$ を適当に定めれば，$y_n = \mathbf{u}\cdot\mathbf{x}_n$ という式で1次元への次元削減ができることが分かりました．
では，$\mathbf{u}$ はどのように定めればよいでしょうか？

主成分分析の目的は，「変換後のデータの分散がなるべく大きくなるように（元のデータの分散を保つように）」することでした．$\mathbf{u}$の向きを変えると，変換後の値 $y_n$ の分散の大きさも変化します．
以下の図(a), (b), (c) は，3種類の $\mathbf{u}$ で2次元のデータを1次元に変換した様子と，変換後の値の分散を示しています．
右図は横軸に $y_n$ の値をとっており，`variance` の値が $y_n$ の分散です．


<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/pcafig03.png">

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/pcafig04.png">

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/pcafig05.png">

図を見ると，データの散らばりが大きい方向を $\mathbf{u}$ が向いたときに，変換後の分散が大きくなっていることがわかります．
直感的な説明になりますが，変換後の値だけを見て元のデータの様子を想像するなら，分散が大きい方が，個々の値の違いがわかりやすくてうれしいですね．

ここまで，データを1次元に次元削減する場合限定で，主成分分析の考え方を説明してきました．
次のセクションでは，このような主成分分析の問題をきちんと定式化して，その解がどのようなものかを説明します．

#### 主成分分析の問題設定とその解（1次元にする場合）

---

［主成分分析の問題設定（1次元にする場合）］

$N$個の$D$次元ベクトルから成るデータ

$$
\{ \mathbf{x}_n \in {\cal R}^{D} | n = 1, 2, \ldots, N\}
$$

が与えられるとする．このデータの平均は $\mathbf{0}$ である，つまり，

$$
\frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_{n} = \mathbf{0}
$$

と仮定する（$\mathbf{0}$でない場合は，平均を求めてそれをデータから差し引いたものをあらためて学習データとすればよい）．
大きさ $1$ の $D$ 次元ベクトル $\mathbf{u}$ を用いて，次式によって $D$次元ベクトル  $\mathbf{x}$ を1つの実数値 $y_n$ に変換することを考える．

$$
y_n = \mathbf{u}\cdot \mathbf{x}_n \quad (n = 1, 2, \ldots, N)
$$

ベクトル $\mathbf{u}$ を，大きさ1($\Vert\mathbf{u}\Vert = 1$)の任意のベクトルの中で $y_n$の分散が最大となるように定めたい．

---


これが主成分分析の問題設定です．
この問題の解，つまり，$y_n$ の分散を最大にする $\mathbf{u}$ は，「**データ $\{\mathbf{x}_n\}$ の分散共分散行列を $V$ とするとき，その固有ベクトルのうち，最大の固有値に対応するもの**」となります（導出過程は後述）．

ここで，**分散共分散行列** （「共分散行列」ともいう） は，次のように定められます：

$\mathbf{x}_n$($n=1,2,\ldots,N$)を $D\times 1$ 行列（$D$次元の列ベクトル）とするとき，
その分散共分散行列 $V$ は

$$
V = \frac{1}{N}\sum_{n=1}^{N}(\mathbf{x}_{n} - \bar{\mathbf{x}})(\mathbf{x}_{n} - \bar{\mathbf{x}})^{\top}
$$

という $D\times D$ 行列です．$V$ の $(i,j)$ 要素 $V_{i,j}$ は

$$
V_{i,j} = \frac{1}{N}\sum_{n=1}^{N}(x_{n,i}-\bar{x}_{i})(x_{n,j}-\bar{x}_{j})  \qquad (i, j = 1, 2, \ldots, D)
$$

という式で表されますが，これは $\mathbf{x}_n$ の $i$ 番目の要素と $j$ 番目の要素の間の **共分散** です（注）．

今の問題設定では，$\mathbf{x}_n$ の平均は $\bar{\mathbf{x}} = \mathbf{0}$ としていますので，$V = \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_{n}\mathbf{x}_{n}^{\top}$ となります．
まとめると，ここに示した主成分分析の解のベクトル $\mathbf{u}$ は，次のステップで求められます．

1. データから $V$ を計算する
1. その固有値固有ベクトルを求める
1. 最大の固有値に対応する固有ベクトルを $\mathbf{u}$ とする

※注: ここでは「分散共分散行列」や「共分散」についてはこれ以上は深入りしません．「データ分析」を履修したひとは，「相関係数」の話で登場していますので，そのときの資料が参考になるかもしれません
（2022年度「データ分析」[ex05notebookA.ipynb](https://github.com/takatakamanbou/Data/blob/2022/ex05notebookA.ipynb)）．
また，ここでは標本分散で定義していますが，不偏分散で（または期待値を使って）定義することもあります．


#### ［発展］主成分分析の問題の解の導出（1次元にする場合）

上記の解の導出過程の概略を以下に示します．

$\mathbf{x}_n$ の平均が $\mathbf{0}$ なので，

$$
\frac{1}{N}\sum_{n=1}^{N}y_n = \frac{1}{N}\sum_{n=1}^{N}\mathbf{u}\cdot\mathbf{x}_{n} = \mathbf{u}\cdot\left( \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_n\right) = \mathbf{u}\cdot\mathbf{0} = 0
$$

つまり，$y_n$ の平均も $0$ です．そのため，$y_n$ の分散は

$$
\frac{1}{N}\sum_{n=1}^{N}y_n^2
$$

と表せます．この式は，次のように変形できます．

$$
\begin{aligned}
\frac{1}{N}\sum_{n=1}^{N}y_n^2 &= \frac{1}{N}\sum_{n=1}^{N} (\mathbf{u}\cdot\mathbf{x}_n)(\mathbf{u}\cdot\mathbf{x}_n) \\
&= \frac{1}{N}\sum_{n=1}^{N} \left(\mathbf{u}^{\top}\mathbf{x}_n\right) \left(\mathbf{u}^{\top}\mathbf{x}_n\right)^{\top}\\
&= \frac{1}{N}\sum_{n=1}^{N} \mathbf{u}^{\top}\mathbf{x}_n\mathbf{x}_n^{\top} \mathbf{u}\\
&= \mathbf{u}^{\top}\left( \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_n\mathbf{x}_n^{\top}\right) \mathbf{u} = \mathbf{u}^{\top}V\mathbf{u}
\end{aligned}
$$
<!---
ただし，
$$
V = \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_n\mathbf{x}_n^{\top}
$$
は，$\mathbf{x}_n$ の平均が $\mathbf{0}$ であるため，$\mathbf{x}_n$の分散共分散行列に一致します．
--->


いま求めたいのは，$y_n$ の分散 $\mathbf{u}^\top V\mathbf{u}$ を最大にするベクトル $\mathbf{u}$ です．ただし，$\mathbf{u}$ には $\Vert\mathbf{u}\Vert = 1$ という条件があります．
このような制約条件付きの最適化問題を解く定番の手法は，「ラグランジュの未定乗数法」です（注）．
この問題の場合，ラグランジュ乗数を $\lambda$ として，
$$
L = \mathbf{u}^{\top}V\mathbf{u} - \lambda(\mathbf{u}^{\top}\mathbf{u} - 1)
$$
とおけば，$\frac{\partial L}{\partial \mathbf{u}} = \mathbf{0}$ を満たす $\mathbf{u}$ が解の候補となります．

<span style="font-size: 75%">
※注: 「ラグランジュの未定乗数法」は，大学初年次の微積分で学んでいる...かも．興味のあるひとは数学の参考書を調べてね．
</span>

ここで，
$$
\frac{\partial L}{\partial \mathbf{u}} = 2V\mathbf{u} - 2\lambda\mathbf{u}
$$
なので（注），解は
$$
V\mathbf{u} = \lambda\mathbf{u}
$$
を満たさねばなりません．この式より，$V$ の単位固有ベクトルが解の候補であることがわかります．この式を $L$ の式に代入すると，
$$
L = \lambda\mathbf{u}^\top\mathbf{u} - \lambda\cdot 0 = \lambda
$$
となります．解の候補は $V$ の単位固有ベクトルであり，そのときの $L$ の値が固有値に等しいということですので，$L$ を最大にする解は，$V$の最大固有値に対する固有ベクトルとなります．

<span style="font-size: 75%">
※注: $\frac{\partial L}{\partial \mathbf{u}}$ は，「$L$を $\mathbf{u}$ の各要素で偏微分したものをならべたベクトル」です．
</span>

---
### 2次元以上にする場合の主成分分析


> **主成分分析** (Principal Component Analysis, PCA) は，多変量（多次元）のデータが与えられたときに，そのデータが含む情報をなるべく損なわずにより低次元のデータに変換するための多変量解析手法です．
$D$個の変数から成る（$D$次元の）データが与えられたときに，それを$H \leq D$ 個の変数で表す（$H$次元に変換する）ことを目的とします．この変換によってデータの次元数が削減されますので，このような操作は **次元削減** (Dimensionaliry Reduction) と呼ばれることもあります．主成分分析は，次元削減のための手法の一つです．


$2 \leq H \leq D$ の場合の主成分分析について説明します．


まずは $D = 3, H = 2$ の場合で説明します．主成分分析では線形変換を考えますので，$D = 3$ 次元のデータ $\mathbf{x} = (x_1, x_2, x_3)$ を $H = 2$ 次元にするということは，適当な $2 \times 3$ 行列 $W$ を用いて

$$
\mathbf{y} = W\mathbf{x}
$$

とすることを意味します．$\mathbf{y} = (y_1, y_2)$ が変換後のデータです．
このとき，$W$ の $(i,j)$ 要素を $w_{i,j}$ を表すと，

$$
\left\{
\begin{aligned}
y_1 &= w_{1,1}x_1 + w_{1,2}x_2 + w_{1,3}x_3 = \mathbf{w}_{1}\cdot\mathbf{x}\\
y_2 &= w_{2,1}x_1 + w_{2,2}x_2 + w_{2,3}x_3 = \mathbf{w}_{2}\cdot\mathbf{x}\\
\end{aligned}
\right.
$$

と書けます．ただし，$\mathbf{w}_1 = (w_{1,1},w_{1,2},w_{1,3})$ と $\mathbf{w}_2 = (w_{2,1},w_{2,2},w_{2,3})$ はいずれも $D$ 次元ベクトルです（下図も参照）．

このように表せることから分かるように，1次元の場合には$D$次元のベクトルを1つだけ求めていたものが，2次元に次元削減したい場合は$D$次元ベクトルを2つ求めることになります．

<img width=480 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/dimreduction.png">

一般化すると，$D$ 次元のデータを $H$ 次元に次元削減したい場合は，$H\times D$ 行列 $W$ を適切に定めることになります．つまり，$D$ 次元ベクトルを $H$ 個定めることになります．


主成分分析では，「データが含む情報をなるべく損なわずにより低次元のデータに変換する」ために，「**変換後のデータの分散がなるべく大きくなるように（元のデータの分散を保つように）**（注）」します．$H = 1$ の場合，これを実現するには，「データの分散共分散行列の最大固有値に対応する固有ベクトル」を選べばよい，ということでした．

$H = 2$ の場合は，$\mathbf{w}_1$を「データの分散共分散行列の最大固有値に対応する固有ベクトル」とした上で，$\mathbf{w}_2$ を「データの分散のうち $\mathbf{w}_1$ で説明しきれなかった分を最もよく説明できる向き」にとります．
これは，$\mathbf{w}_2$ を，「$\mathbf{w}_1$ に直交しかつ大きさが $1$ のベクトルの中で，$y_2 = \mathbf{w}_2\cdot\mathbf{x}$ の分散が最も大きくなるものとする」ことを意味し，$\mathbf{w}_2$ を「データの分散共分散行列の2番目に大きな固有値に対応する固有ベクトル」の向きにとればよいことがわかっています（数学的にきちんとした議論は省略します）．

<span style="font-size: 75%">
※注: 一般の（$H \geq 2$の）場合，$\mathbf{y}$ の各要素間の共分散が $0$ になるように（分散共分散行列が対角行列になるように）かつ各要素の分散（分散共分散行列の対角要素）の和が最大になるようにします．
</span>

下図は，$D=3$ の場合を図示しています．左の図のピンクのベクトルは $\mathbf{w}_1$ を表しており，このデータはこのベクトルの向きに最も分散が大きくなっています．
中央の緑色の平面は，原点を通りかつピンクのベクトルと直交しています．$\mathbf{w}_2$ は，この平面内の様々な向きのベクトルたち（?で示されているもの）の中から選ぶことになります．このとき，下図右のように，この緑の平面上でデータ点を眺めたときに分散が最も大きくなる方向を選べば，$y_2$の分散を最大化することになります．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/pcafig06.png">

$H \geq 3$ の場合も，$\mathbf{w}_2$ と同じように一つずつベクトルを追加していくと考えることができます．すなわち，$\mathbf{w}_3$ は，$\mathbf{w}_{1}, \mathbf{w}_2$ の両方に直交する大きさベクトルの中で分散最大の方向のものとし，$\mathbf{w}_4$ は $\mathbf{w}_{1}, \mathbf{w}_2, \mathbf{w}_3$ の全てに直交し...，という具合です．そして，結論として，次のことがいえます．

---

［主成分分析］

$N$個の$D$次元ベクトルから成るデータ

$$
\{ \mathbf{x}_n \in {\cal R}^{D} | n = 1, 2, \ldots, N\}
$$

を，$H \times D$ 行列 $W$ によって

$$
\mathbf{y}_n = W\mathbf{x}_n
$$

と $H$ 次元に変換することを考える．ただし，$\{\mathbf{x}_n\}$ の平均は $\mathbf{0}$ とする．
このとき，変換後のデータの分散がなるべく大きくなるように（元のデータの分散を保つように）したいならば，$W$ を次のように構成すればよい．

1. データの分散共分散行列の固有値と固有ベクトルを求める．固有値を $\lambda_1 \geq \lambda_2 \geq \lambda_3 \geq \cdots \geq \lambda_D$ とし，それぞれに対応する固有ベクトルを $\mathbf{u}_1, \mathbf{u}_2, \ldots, \mathbf{u}_D$ とおく（注）．
2. $\mathbf{u}_1, \mathbf{u}_2, \ldots, \mathbf{u}_H$ を並べてできる $H\times D$ 行列を $W$ とする（要素の並べ方は上で説明した通り）．


<span style="font-size: 75%">
※注1: 分散共分散行列は対称行列であるため，固有値は必ず実数となります．また，$0$ 以上となることもいえます．
</span>

---

---
### 主成分と主成分スコア

$D$ 個の変数から成るベクトル $\mathbf{x}$ を上記の手順で $H$ 個の変数から成るベクトル $\mathbf{y}$ に変換したとき，得られる $H$ 個の変数のうち $h$ 番目のもの（$\mathbf{y}$ の $h$ 番目の要素，$h = 1, 2, \ldots, H$）を，**第 $h$ 主成分** と呼びます．
上記の説明から分かるように，番号の小さい主成分ほど元のデータの分散をよく表す変数となっています．
直感的な言い方になりますが，番号の小さい主成分ほど元のデータの散らばりをよく表す重要な変数といえます．
また，一つのデータを変換して得られる$H$個の主成分のうち，第 $h$ 主成分の変数の値のことを，**第 $h$ 主成分スコア** と呼びます． 


